
## Chain of Thought (CoT) Creation

In this notebook, we aim to create a Chain of Thought (CoT) for clinical trial studies using HuggingFace's transformers and other relevant libraries. Below is a step-by-step guide to our workflow:

1. **Setup and Initialization**:
  - We initialize the ChromaDB client and load the SentenceTransformer model for encoding queries.
  - We define a function to fix invalid JSON strings, which is crucial for handling the metadata of clinical trial studies.

2. **Retrieving Relevant Studies**:
  - We define a function `retrieve_relevant_studies` that queries the ChromaDB collection to find studies relevant to a given query, excluding the study already present in the query.

3. **Crafting Context from Studies**:
  - We define a function `craft_context_from_studies_documents` to create a context string from the documents of related studies. This context is used to provide examples in the CoT creation process.

4. **Generating Messages for CoT**:
  - We define a function `get_messages_for_create_CoT` that generates the system and user messages required for creating a CoT. These messages include the study title, description, and desired criteria.

5. **Prompt Creation**:
  - We define a function `get_prompt_from_studies` that uses the above functions to generate the complete prompt for a given study. This prompt includes the context from related studies and the task instructions for generating the CoT.

6. **Model Inference**:
  - We load the HuggingFace model and tokenizer, and define a function `pipe` to generate the CoT using the model. The function takes the generated messages as input and returns the model's output.
  - For Gemini, can use the function in the gemini section to generate the CoT.

By following this workflow, we can systematically generate a Chain of Thought for clinical trial studies, leveraging the capabilities of HuggingFace's transformers and other relevant tools.
```

In [14]:
import os
# os.environ['HF_HUB_DISABLE_SSL_VERIFY'] = '1'
# os.environ['HF_HUB_HTTP_TIMEOUT'] = "1"
!export HF_HUB_HTTP_TIMEOUT=1
import chromadb
from sentence_transformers import SentenceTransformer
from client import Client
from prompt_gen import PromptGen

client = chromadb.PersistentClient(path="./clinical_trials_chroma")
embed_model = SentenceTransformer("malteos/scincl")
collection = client.get_or_create_collection("clinical_trials_studies")


client = Client(
  client=client,
  collection=collection,
  embed_model=embed_model
)

prompt_gen = PromptGen(
  client=client
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
print(
    client.retrieve_relevant_studies("A Prospective, Controlled, Randomized, Multicenter Study of the Efficacy of an Autophagy Inhibitor (GNS561), an Anti-NKG2A (Monalizumab) and an Anti-C5aR (Avdoralimab) Compared to the Standard of Care in Patients With Advanced or Metastatic Cancer and SARS-CoV-2 (COVID-19) Infection.\nDescription: #Study Description \nBrief Summary \nA prospective, controlled, randomized, multicenter study whose goal is to compare the efficacy of an autophagy inhibitor (GNS561), an anti-NKG2A (monalizumab) and an anti-C5aR (avdoralimab) versus standard of care in patients with advanced or metastatic cancer who have Sars-CoV-2 infection not eligible to a resuscitation unit.\n\nAccording to their severity level at the time of enrolment, eligible patients will be randomized into 2 different cohorts:\n\n* COHORT 1 (mild symptoms or asymptomatic): GNS561 vs anti-NKG2A vs standard of care (randomization ratio 1:1:1).\n* COHORT 2 (moderate/severe symptoms): anti-C5aR vs standard of care (randomization ratio 1:1).\n\n#Intervention \n- DRUG : Autophagy inhibitor (GNS651) \n\t- Cohort 1 (arm B): 200mg q.d. orally for 10 consecutive days. If for any reason a treatment is not given within the allowed treatment window (± 12h) it will be cancelled (i.e., missed for that time point), and treatment will be resumed at the next dosing day.\n- OTHER : Standard of care \n\t- In cohorts 1 and 2, patients allocated in the standard of care arms should receive best supportive care, as per the investigator\'s discretion and the local routine practices. With regards to the respiratory symptoms and medical resoures at investigational site, the following should be given according to the patient\'s condition: oxygen supplementation, non-invasive ventilation, invasive ventilation, antibiotherapy, vasopressor support, renal replacement therapy, or extracorporeal membrane oxygenation.\n\nAdditional care and medications should be administered in the patient\'s best interest.\n- DRUG : Avdoralimab \n\t- Cohorte 2 (arm H): 500mg, intravenously, at Day 1 then 200mg once daily every 2 days during 14 Days\n- DRUG : Monalizumab \n\t- Cohorte 2 (arm G) : 50mg (flat dose),intravenously, single infusion at Day 1.\nDesired criteria: Exclusion Criteria: E8. Patient has known active hepatitis B (chronic or acute; defined as having a positive hepatitis B surface antigen [HBsAg] test at screening), known active hepatitis C (Patients positive for hepatitis C virus (HCV) antibody are eligible only if PCR is negative for HCV RNA at screening) or known Human Immunodeficiency Virus (HIV) infection (HIV 1/2 antibodies).", 
                              "NCT04333914"))



0
[]


In [14]:
# from vllm import LLM, SamplingParams
# from vllm.sampling_params import GuidedDecodingParams
# from pydantic import BaseModel
# from typing import List, Optional
# from enum import Enum

# class SexEnum(str, Enum):
#     MALE = "MALE"
#     FEMALE = "FEMALE"
#     ALL = "ALL"

# class AgeGroupEnum(str, Enum):
#     CHILD = "CHILD"
#     ADULT = "ADULT"
#     OLDER_ADULT = "OLDER_ADULT"

# class Age(BaseModel):
#     Min: Optional[int] = None
#     Max: Optional[int] = None
#     AgeGroup: List[AgeGroupEnum]

# class EC(BaseModel):
#     InclusionCriteria: List[str] = []
#     ExclusionCriteria: List[str] = []

# json_schema = EC.model_json_schema()
# model_id = "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16"
# guided_decoding_params = GuidedDecodingParams(json=json_schema)
# sampling_params = SamplingParams(guided_decoding=guided_decoding_params, max_tokens=4096, temperature=0.2, top_k=50)
# llm = LLM(model=model_id, max_model_len=10000)
# import json
# def extract_json(output):
#         try:
#             return json.loads(output)
#         except:
#             print(output)
#             return None
# def batch_extract_ec(raw_ecs: List[str], llm):
#     base_prompt = """Using the following eligibility criteria details, generate a clean and structured JSON output that adheres to the schema provided. Do not rewrite the criteria. Just recite it in a structured JSON format."""
#     raw_ecs = [i.split("#Sex :")[0] if i is not None else "" for i in raw_ecs]
#     print(raw_ecs[0])
#     prompts = [f"{base_prompt}\n{ec}".replace("(Child: birth-17, Adult: 18-64, Older Adult: 65+)", "") for ec in raw_ecs]
#     print(prompts[0])
#     outputs = llm.generate(
#         prompts=prompts,
#         sampling_params=sampling_params
#     )
#     return [extract_json(output.outputs[0].text) for output in outputs]
    



## Using Llama 3.1

In [4]:

# !pip show vllm


from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

model_id = "neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16"
number_gpus = 1

tokenizer = AutoTokenizer.from_pretrained(model_id)

llm = LLM(model=model_id, tensor_parallel_size=number_gpus, max_model_len=20000)

def pipe(messages):
    sampling_params = SamplingParams(temperature=0, top_p=0.9, max_tokens=4096)
    prompts = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    outputs = llm.generate(prompts, sampling_params)
    return [i.outputs[0].text for i in outputs]

Loading fast tokenizer from /home/swiss/.cache/huggingface/hub/models--neuralmagic--Meta-Llama-3.1-8B-Instruct-quantized.w8a16/snapshots/38e03ba250017bf8ed3eeecd3a744e21f6b994a9/tokenizer.json
INFO 01-01 17:51:28 config.py:478] This model supports multiple tasks: {'reward', 'generate', 'classify', 'embed', 'score'}. Defaulting to 'generate'.
INFO 01-01 17:51:28 llm_engine.py:249] Initializing an LLM engine (v0.6.5) with config: model='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16', speculative_config=None, tokenizer='neuralmagic/Meta-Llama-3.1-8B-Instruct-quantized.w8a16', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=20000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=compressed-tensors, enforce_eager=False, kv_cache_dtype=auto, quantization_param_pat

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 01-01 17:51:38 model_runner.py:1097] Loading model weights took 8.4927 GB
INFO 01-01 17:51:43 worker.py:241] Memory profiling takes 5.60 seconds
INFO 01-01 17:51:43 worker.py:241] the current vLLM instance can use total_gpu_memory (23.68GiB) x gpu_memory_utilization (0.90) = 21.32GiB
INFO 01-01 17:51:43 worker.py:241] model weights take 8.49GiB; non_torch_memory takes 0.04GiB; PyTorch activation peak memory takes 2.06GiB; the rest of the memory reserved for KV Cache is 10.73GiB.
INFO 01-01 17:51:43 gpu_executor.py:76] # GPU blocks: 5491, # CPU blocks: 2048
INFO 01-01 17:51:43 gpu_executor.py:80] Maximum concurrency for 20000 tokens per request: 4.39x
INFO 01-01 17:51:46 model_runner.py:1413] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 01-01 17:51:46 model_runner.py:1417] If out-of-memory error occurs during cudagraph capture

In [13]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm

# Load the JSON data
ravis_dataset = load_dataset("ravistech/clinical-trial-llm-cancer-restructure")
## convert the dataset to a pandas DataFrame
ravis_df = pd.DataFrame(ravis_dataset['train'])
# Initialize an empty DataFrame
### =['encoded_related_studies', 'title', 'description', 'desired_criteria', 'messages'])
## add columns to the ravis_df
ravis_df["encoded_related_studies"] = None
ravis_df["title"] = None
ravis_df["description"] = None
ravis_df["desired_criteria"] = None
ravis_df["messages"] = None
ravis_df["parsed_ec"] = None
import re
import json
import pandas as pd

def adjust_indentation_for_colons(line_objs):
    i = 0
    n = len(line_objs)
    while i < n:
        this_line = line_objs[i]
        text = this_line['text'].rstrip()
        indent = this_line['indent']

        if text.endswith(':'):
            j = i + 1
            while j < n:
                next_line = line_objs[j]
                next_text = next_line['text']
                next_indent = next_line['indent']

                if next_text.endswith(':'):
                    break
                if next_indent < indent:
                    break
                if next_indent == indent:
                    line_objs[j]['indent'] = indent + 2
                j += 1

        i += 1
    return line_objs

def parse_lines_to_tree(lines):
    root = {'text': 'ROOT', 'indent': -1, 'children': []}
    stack = [root]

    for line in lines:
        line_indent = line['indent']
        line_text = line['text']

        while stack and stack[-1]['indent'] >= line_indent:
            stack.pop()

        node = {'text': line_text, 'indent': line_indent, 'children': []}
        stack[-1]['children'].append(node)
        stack.append(node)

    return root

def gather_all_texts(node, combined_list):
    combined_list.append(f"* {node['text'].strip()}")
    for c in node['children']:
        gather_all_texts(c, combined_list)

def flatten_tree_with_rules(node, current_line, depth, max_depth=3, max_siblings=5):
    results = []

    if depth >= max_depth and node['children']:
        combo = []
        gather_all_texts(node, combo)
        return [f"{current_line} {' '.join(combo)}"]

    if not node['children']:
        return [current_line]

    num_children = len(node['children'])
    if num_children > max_siblings and depth > 1:
        bullets = [f"* {child['text'].strip()}" for child in node['children']]
        merged = " ".join(bullets)
        return [f"{current_line} {merged}"]

    for child in node['children']:
        child_text = f"{current_line} {child['text'].strip()}"
        child_flat = flatten_tree_with_rules(
            child, child_text, depth + 1, max_depth=max_depth, max_siblings=max_siblings
        )
        results.extend(child_flat)

    return results

def parse_criteria_final(text):
    eligibility_sections = text.split("#Eligibility Criteria:")
    parsed_results = []

    for section in eligibility_sections:
        section = section.strip()
        if not section:
            continue

        block_data = {
            "Inclusion Criteria": [],
            "Exclusion Criteria": [],
            "Sex": None,
            "Minimum Age": None,
            "Maximum Age": None,
            "Age Groups": [],
            "Accepts Healthy Volunteers": None
        }

        inclusion_match = re.search(
            r"Inclusion\s*criteria\s*([\s\S]*?)(?=Exclusion\s*criteria|##Sex)",
            section, re.IGNORECASE
        )
        if inclusion_match:
            inclusion_block_text = inclusion_match.group(1)
            inclusion_lines = inclusion_block_text.split('\n')
            inclusion_line_objs = []
            for ln in inclusion_lines:
                stripped_ln = ln.strip()
                if stripped_ln and stripped_ln != ":":
                    indent_len = len(ln) - len(ln.lstrip())
                    inclusion_line_objs.append({'indent': indent_len, 'text': stripped_ln})

            inclusion_line_objs = adjust_indentation_for_colons(inclusion_line_objs)
            inclusion_root = parse_lines_to_tree(inclusion_line_objs)
            for top_node in inclusion_root['children']:
                flattened = flatten_tree_with_rules(
                    top_node, top_node['text'].strip(), depth=0
                )
                block_data["Inclusion Criteria"].extend(flattened)

        exclusion_match = re.search(
            r"Exclusion\s*criteria\s*([\s\S]*?)(?=##Sex)",
            section, re.IGNORECASE
        )
        if exclusion_match:
            exclusion_block_text = exclusion_match.group(1)
            exclusion_lines = exclusion_block_text.split('\n')
            exclusion_line_objs = []
            for ln in exclusion_lines:
                stripped_ln = ln.strip()
                if stripped_ln and stripped_ln != ":":
                    indent_len = len(ln) - len(ln.lstrip())
                    exclusion_line_objs.append({'indent': indent_len, 'text': stripped_ln})

            exclusion_line_objs = adjust_indentation_for_colons(exclusion_line_objs)
            exclusion_root = parse_lines_to_tree(exclusion_line_objs)
            for top_node in exclusion_root['children']:
                flattened = flatten_tree_with_rules(
                    top_node, top_node['text'].strip(), depth=0
                )
                block_data["Exclusion Criteria"].extend(flattened)

        sex_match = re.search(r"##Sex\s*:\s*(.+)", section, re.IGNORECASE)
        if sex_match:
            block_data["Sex"] = sex_match.group(1).strip()

        min_age_match = re.search(r"Minimum\s+Age\s*:\s*([\d\w\s]+)", section, re.IGNORECASE)
        if min_age_match:
            block_data["Minimum Age"] = min_age_match.group(1).strip()

        max_age_match = re.search(r"Maximum\s+Age\s*:\s*([\d\w\s]+)", section, re.IGNORECASE)
        if max_age_match:
            block_data["Maximum Age"] = max_age_match.group(1).strip()

        age_group_match = re.search(r"Age Group.*:\s*(.+)", section, re.IGNORECASE)
        if age_group_match:
            raw_groups = age_group_match.group(1)
            age_groups = [g.strip() for g in raw_groups.split(',') if g.strip()]
            block_data["Age Groups"] = age_groups

        hv_match = re.search(r"##Accepts Healthy Volunteers\s*:\s*(\w+)", section, re.IGNORECASE)
        if hv_match:
            hv_value = hv_match.group(1).strip().lower()
            if hv_value == "yes":
                block_data["Accepts Healthy Volunteers"] = True
            elif hv_value == "no":
                block_data["Accepts Healthy Volunteers"] = False
            else:
                block_data["Accepts Healthy Volunteers"] = None

        parsed_results.append(json.dumps(block_data))

    return parsed_results

for i in tqdm(range(len(ravis_df))):
    ravis_df.loc[i, "parsed_ec"] = parse_criteria_final(ravis_df.loc[i, "criteria"])

## drop the row if the parsed_ec Inclusion Criteria and Exclusion Criteria are empty
ravis_df = ravis_df[ravis_df["parsed_ec"].apply(lambda x: any(json.loads(x[0])["Inclusion Criteria"]) or any(json.loads(x[0])["Exclusion Criteria"]))]

## drop the row if the parsed_ec Inclusion Criteria and Exclusion Criteria are empty and more than 1
ravis_df = ravis_df[ravis_df["parsed_ec"].apply(lambda x: len(x) == 1)]



100%|██████████| 31938/31938 [00:07<00:00, 4363.77it/s]


In [17]:
## save the ravis_df to a temp pickle file
# ravis_df.to_pickle("ravis_df.pkl")
# load the ravis_df from the pickle file
# import pandas as pd
ravis_df = pd.read_pickle("ravis_df.pk")


,metadata,data,criteria,__index_level_0__
0,"metadata\n{\n""NCT_ID"" :NCT01443390,\n""Brief_Ti...",Study Description \nBrief Summary \nWhen a pat...,Accepts Healthy Volunteers: No\n,27943
1,"metadata\n{\n""NCT_ID"" :NCT05243823,\n""Brief_Ti...",Study Description \nBrief Summary \nThe study ...,Accepts Healthy Volunteers: No\n,23427
2,"metadata\n{\n""NCT_ID"" :NCT00538733,\n""Brief_Ti...",Study Description \nBrief Summary \nStudy Obje...,Accepts Healthy Volunteers: No\n,33867
3,"metadata\n{\n""NCT_ID"" :NCT00188162,\n""Brief_Ti...",Study Description \nBrief Summary \nThe aim is...,Accepts Healthy Volunteers: No\n,8790
4,"metadata\n{\n""NCT_ID"" :NCT00625768,\n""Brief_Ti...",Study Description \nBrief Summary \nTo determi...,Accepts Healthy Volunteers: No\n,20414
...,...,...,...,...
31802,"metadata\n{\n""NCT_ID"" :NCT00320827,\n""Brief_Ti...",Study Description \nBrief Summary \nThis is a ...,Accepts Healthy Volunteers: No\n,6265
31803,"metadata\n{\n""NCT_ID"" :NCT00557492,\n""Brief_Ti...",Study Description \nBrief Summary \nThis study...,Accepts Healthy Volunteers: No\n,11284
31804,"metadata\n{\n""NCT_ID"" :NCT02671097,\n""Brief_Ti...",Study Description \nBrief Summary \nEvaluation...,Accepts Healthy Volunteers: Yes\n,38158
31805,"metadata\n{\n""NCT_ID"" :NCT02273791,\n""Brief_Ti...",Study Description \nBrief Summary \nEvaluation...,Accepts Healthy Volunteers: No\n,860


In [19]:
## re-run the batch_extract_ec function on the unparsed_df
# from tqdm import tqdm

# batch_size = 100
# for i in tqdm(range(0, len(unparsed_df), batch_size)):
#     raw_ecs = unparsed_df.iloc[i:i+batch_size]["criteria"].tolist()
#     print(raw_ecs)
#     parsed_ecs = batch_extract_ec(raw_ecs, llm)
#     print(parsed_ecs)
#     unparsed_df.loc[i:i+batch_size-1, "parsed_ec"] = parsed_ecs


In [15]:
from tqdm import tqdm
def get_demographic_criteria_message(sex, min_age, max_age, age_groups):
    message = ""
    if sex:
        message += f"Sex: {sex} "
    if min_age:
        message += f"with a minimum age of {min_age} "
    if max_age:
        message += f"and a maximum age of {max_age} "
    if age_groups:
        message += f"and the following age groups (Child: birth-17, Adult: 18-64, Older Adult: 65+): {', '.join(age_groups)} "
    return message


for i, row in tqdm(ravis_df.iterrows(), total=len(ravis_df)):
    # print(f"Processing {study}")
    ## select metadata	data	criteria
    study = row[["metadata", "data", "parsed_ec"]]
    info_for_prompt = prompt_gen.get_info_for_prompt_gen(study)
    if info_for_prompt:
        encoded_related_studies, title, description, desired_criteria = info_for_prompt
        inclusion_criteria = desired_criteria.get("Inclusion Criteria", [])
        exclusion_criteria = desired_criteria.get("Exclusion Criteria", []) 
        sex = desired_criteria.get("Sex", None)
        min_age = desired_criteria.get("Minimum Age", None)
        max_age = desired_criteria.get("Maximum Age", None)
        age_groups = desired_criteria.get("Age Groups", [])
        accepts_healthy_volunteers = desired_criteria.get("Accepts Healthy Volunteers", None)
        inclusion_criteria_messages = [
            f"Inclusion Criteria: {criteria}" for criteria in inclusion_criteria
        ]
        exclusion_criteria_messages = [
            f"Exclusion Criteria: {criteria}" for criteria in exclusion_criteria
        ]
        get_demographic_criteria_messages = [
            get_demographic_criteria_message(sex, min_age, max_age, age_groups)
        ]
        ## only accepts_healthy_volunteers true or false strict
        accepts_healthy_volunteers_message = []
        if accepts_healthy_volunteers is not None:
            if accepts_healthy_volunteers == True:
                message = "Accepts Healthy Volunteers: Yes"
            else:
                message = "Accepts Healthy Volunteers: No"
            accepts_healthy_volunteers_message = [message]
        messages = inclusion_criteria_messages + exclusion_criteria_messages + get_demographic_criteria_messages + accepts_healthy_volunteers_message
        
        # Add the data to the DataFrame exsisting ravis_df
        ravis_df.at[i, "encoded_related_studies"] = encoded_related_studies
        ravis_df.at[i, "title"] = title
        ravis_df.at[i, "description"] = description
        ravis_df.at[i, "messages"] = messages
        
        if i % 1000 == 0:
            print(f"Prompt: {messages}")
        
## save the ravis_df to a temp pickle file
ravis_df.drop(columns=["__index_level_0__", "desired_criteria", "data"], inplace=True)
ravis_df.to_pickle("ravis_df.pkl")

  0%|          | 10/31936 [00:00<12:46, 41.67it/s] 

3
Prompt: ['Inclusion Criteria: * Willing and able to give written informed consent', 'Inclusion Criteria: * Histologic or cytologic diagnosis of pancreas adenocarcinoma advanced or recurrent (stage III or IV) that is unresectable; histologic or cytologic pathology from any prior surgery is sufficient for diagnosis', 'Inclusion Criteria: * Must have measurable disease by modified WHO criteria', 'Inclusion Criteria: * White blood cells (WBC) >= 2000/uL', 'Inclusion Criteria: * Absolute neutrophil count (ANC) >= 1500/uL', 'Inclusion Criteria: * Platelets >= 100 x 10^3/uL', 'Inclusion Criteria: * Hemoglobin >= 9 g/dL (>= 80 g/L; may be transfused)', 'Inclusion Criteria: * Creatinine =< 2.0 x upper limit of normal (ULN)', 'Inclusion Criteria: * Aspartate aminotransferase (AST)/alanine aminotransferase (ALT) =< 2.5 x ULN', "Inclusion Criteria: * Bilirubin =< 2.0 x ULN, (except patients with Gilbert's Syndrome, who must have a total bilirubin less than 3.0 mg/dL)", 'Inclusion Criteria: * No 

  0%|          | 28/31936 [00:00<07:46, 68.42it/s]

3
1
3
3
3
3
2
3
3
3
0
0
3
3
3
3
3


  0%|          | 47/31936 [00:00<06:33, 81.08it/s]

...

2
3
3
3
0
1
0
3
0
3
3
3
2
1
3
3
3
3
0


100%|█████████▉| 31918/31936 [05:50<00:00, 91.26it/s]

0
3
3
3
3
3
1
3
0
3
0
3
3
3
0
0
3


100%|██████████| 31936/31936 [05:50<00:00, 91.03it/s]

1
3
3
3
3
3
3
3
0
2


In [13]:
ravis_df


,metadata,criteria,encoded_related_studies,title,description,messages,parsed_ec
0,"{\n""NCT_ID"" : ""NCT01473940"",\n""Brief_Title"" : ...",#Eligibility Criteria:\nInclusion Criteria:\n\...,<STUDY>\n Related NCT_ID: NCT01893294\n ...,Ipilimumab and Gemcitabine for Advanced Pancre...,#Study Description \nBrief Summary \nThis phas...,[Inclusion Criteria: * Willing and able to giv...,"[{""Inclusion Criteria"": [""* Willing and able t..."
1,"{\n""NCT_ID"" : ""NCT01646697"",\n""Brief_Title"" : ...",#Eligibility Criteria:\nInclusion Criteria:\n\...,<STUDY>\n Related NCT_ID: NCT05959213\n ...,Cytopathologic Margin Evaluation in Patients U...,#Study Description \nBrief Summary \nThis stud...,[Inclusion Criteria: * patients undergoing pan...,"[{""Inclusion Criteria"": [""* patients undergoin..."
2,"{\n""NCT_ID"" : ""NCT01771666"",\n""Brief_Title"" : ...",#Eligibility Criteria:\nInclusion Criteria:\n\...,<STUDY>\n Related NCT_ID: NCT05859971\n ...,A Pilot Study to Assess the Utility of Indocya...,#Study Description \nBrief Summary \nThe purpo...,[Inclusion Criteria: * Ability to understand a...,"[{""Inclusion Criteria"": [""* Ability to underst..."
3,"{\n""NCT_ID"" : ""NCT04293094"",\n""Brief_Title"" : ...",#Eligibility Criteria:\nInclusion Criteria:\n\...,<STUDY>\n Related NCT_ID: NCT01363024\n ...,"A Phase 1, Multicenter, Open-label, Dose-Explo...",#Study Description \nBrief Summary \nTo evalua...,[Inclusion Criteria: * Male and female >= 18 y...,"[{""Inclusion Criteria"": [""* Male and female >=..."
4,"{\n""NCT_ID"" : ""NCT01460238"",\n""Brief_Title"" : ...",#Eligibility Criteria:\nInclusion Criteria:\n\...,,Lipoprotein Lipase Expression in Chronic Lymph...,#Study Description \nBrief Summary \nThe inves...,[Inclusion Criteria: * Diagnosis of CLL as per...,"[{""Inclusion Criteria"": [""* Diagnosis of CLL a..."
...,...,...,...,...,...,...,...
31933,"{\n""NCT_ID"" : ""NCT00270166"",\n""Brief_Title"" : ...",#Eligibility Criteria:\nInclusion Criteria:\n\...,<STUDY>\n Related NCT_ID: NCT00270127\n ...,A Placebo-Controlled Study on the Effect of Ep...,#Study Description \nBrief Summary \nThe purpo...,[Inclusion Criteria: * Confirmed diagnosis of ...,"[{""Inclusion Criteria"": [""* Confirmed diagnosi..."
31934,"{\n""NCT_ID"" : ""NCT02658968"",\n""Brief_Title"" : ...",#Eligibility Criteria:\nInclusion Criteria:\n\...,<STUDY>\n Related NCT_ID: NCT00062894\n ...,A Phase 1 Dose Finding Study of Lutetium (177L...,#Study Description \nBrief Summary \nThis stud...,[Inclusion Criteria: * Male or female aged >=1...,"[{""Inclusion Criteria"": [""* Male or female age..."
31935,"{\n""NCT_ID"" : ""NCT01209598"",\n""Brief_Title"" : ...",#Eligibility Criteria:\nInclusion Criteria:\n\...,<STUDY>\n Related NCT_ID: NCT01143740\n ...,A Phase II Study Of PD0332991 (Palbociclib) in...,#Study Description \nBrief Summary \nThe purpo...,[Inclusion Criteria: * A diagnosis of liposarc...,"[{""Inclusion Criteria"": [""* A diagnosis of lip..."
31936,"{\n""NCT_ID"" : ""NCT00600431"",\n""Brief_Title"" : ...",#Eligibility Criteria:\nInclusion Criteria:\n\...,,Melanocytic-Nevi-Counts During Chemotherapy of...,#Study Description \nBrief Summary \nChanges i...,[Inclusion Criteria: * Patient and/or parents ...,"[{""Inclusion Criteria"": [""* Patient and/or par..."


In [8]:

def pipe_batch(batch_messages):
    sampling_params = SamplingParams(temperature=0, top_p=0.9, max_tokens=4096)
    prompts = [
        tokenizer.apply_chat_template(msg, add_generation_prompt=True, tokenize=False)
        for msg in batch_messages
    ]
    outputs = llm.generate(prompts, sampling_params)
    return [output.outputs[0].text for output in outputs]

batch_size = 200  # Batch size for processing
responses = []

# Iterate through DataFrame and process in batches
batch_messages = []
indices = []

for i, row in tqdm(ravis_df.iterrows(), total=len(ravis_df)):
    encoded_related_studies = row["encoded_related_studies"]
    title = row["title"]
    description = row["description"]
    messages_instruct = [PromptGen.gen_messages(PromptGen.get_messages_for_CoT_huggingface(encoded_related_studies, title, description, message)) for message in row["messages"]]
    for message in messages_instruct:
        batch_messages.append(message)
        indices.append(i)
        # If batch is full, process it
        if len(batch_messages) == batch_size:
            batch_responses = pipe_batch([(i) for i in batch_messages])
            responses.extend(batch_responses)
            batch_messages = []
            indices = []

# Process remaining messages in the last batch
if batch_messages:
    batch_responses = pipe_batch(batch_messages)
    responses.extend(batch_responses)

# Map responses back to rows in the DataFrame
response_dict = {}
for idx, response in zip(indices, responses):
    if idx not in response_dict:
        response_dict[idx] = []
    response_dict[idx].append(response)
ravis_df["response"] = None
ravis_df["response"] = ravis_df.index.map(lambda idx: response_dict.get(idx, []))

# Save the updated DataFrame if needed
ravis_df.to_pickle("ravis_df_with_responses.pkl")

  0%|          | 0/31934 [00:00<?, ?it/s]

WARNING 01-01 17:59:39 scheduler.py:1555] Sequence group 12 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


INFO 01-01 18:05:48 model_runner_base.py:120] Writing input of failed execution to /tmp/err_execute_model_input_20250101-180548.pkl...
INFO 01-01 18:05:49 model_runner_base.py:149] Completed writing input of failed execution to /tmp/err_execute_model_input_20250101-180548.pkl.


  0%|          | 31/31934 [07:27<127:47:50, 14.42s/it]


OutOfMemoryError: Error in model execution (input dumped to /tmp/err_execute_model_input_20250101-180548.pkl): CUDA out of memory. Tried to allocate 534.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 420.62 MiB is free. Including non-PyTorch memory, this process has 23.11 GiB memory in use. Of the allocated memory 21.22 GiB is allocated by PyTorch, with 68.00 MiB allocated in private pools (e.g., CUDA Graphs), and 686.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
ravis_df

## Using Gemini

In [5]:
# import time
# import pandas as pd
# import json
# from datasets import load_dataset
# from tqdm import tqdm
# import vertexai
# from vertexai.batch_prediction import BatchPredictionJob
# import os
# import uuid

# PROJECT_ID = "PROJECT-ID"  # update with Google Cloud project ID
# BUCKET_NAME = "BUCKET-NAME" # create a bucket in Google Cloud Storage first
# OUTPUT_DIR = "./output/"
# CSV_OUTPUT_PATH = "./responses_gemini.csv"
# vertexai.init(project=PROJECT_ID, location="us-central1")

# ravis_dataset = load_dataset("ravistech/clinical-trial-llm-cancer-restructure")

# batch_prompts = []
# info_data = {}  # key is uuid, value is the info for the prompt
# for study in tqdm(ravis_dataset['train']):
#     info_for_prompt = prompt_gen.get_info_for_prompt_gen(study)
    
#     if info_for_prompt:
#         unique_id = str(uuid.uuid4())  # gen uuid for each entries
#         encoded_related_studies, title, description, desired_criteria = info_for_prompt
#         messages = prompt_gen.user_prompt_template(encoded_related_studies, title, description, desired_criteria)

#         request_format = {
#             "id": unique_id,
#             "request": {
#                 "contents": [
#                     {
#                         "role": "user",
#                         "parts": [
#                             {"text": messages}
#                         ]
#                     }
#                 ],
#                 "system_instruction": {
#                     "parts": [
#                         {"text": prompt_gen.system_prompt}
#                     ]
#                 }
#             }
#         }
#         batch_prompts.append(request_format)
        
#         # store info with unique ID for later matching with the response
#         info_data[unique_id] = {
#             "encoded_related_studies": encoded_related_studies,
#             "title": title,
#             "description": description,
#             "desired_criteria": desired_criteria,
#             "messages": messages,
#             "response": None
#         }

# input_jsonl_path = f'gs://{BUCKET_NAME}/prompt_for_batch_gemini_predict.jsonl'
# with open('./prompt_for_batch_gemini_predict.jsonl', 'w') as f:
#     for prompt in batch_prompts:
#         f.write(json.dumps(prompt) + '\n')

# # Upload the JSONL file to GCS
# !gsutil cp ./prompt_for_batch_gemini_predict.jsonl {input_jsonl_path}

# batch_prediction_job = BatchPredictionJob.submit(
#     source_model="gemini-1.5-flash-002",
#     input_dataset=input_jsonl_path,
#     output_uri_prefix=f'gs://{BUCKET_NAME}/output/',
# )

# print(f"Job resource name: {batch_prediction_job.resource_name}")
# print(f"Model resource name with the job: {batch_prediction_job.model_name}")
# print(f"Job state: {batch_prediction_job.state.name}")

# # Check the job status
# while not batch_prediction_job.has_ended:
#     time.sleep(5)
#     batch_prediction_job.refresh()

# # Process the output if the job is successful
# if batch_prediction_job.has_succeeded:
#     print("Job succeeded!")
#     output_location = batch_prediction_job.output_location + "/predictions.jsonl"
#     os.makedirs(OUTPUT_DIR, exist_ok=True)
#     local_output_path = os.path.join(OUTPUT_DIR, "predictions.jsonl")

#     # Download the output file from GCS
#     !gsutil cp {output_location} {local_output_path}
#     print(f"Output file downloaded to: {local_output_path}")
    
#     with open(local_output_path, 'r') as f:
#         for line in f:
#             response_data = json.loads(line)
#             unique_id = response_data.get("id")
#             response = response_data.get("response", {})
            
#             if unique_id in info_data:
#                 info_data[unique_id]["response"] = response

#     df = pd.DataFrame.from_dict(info_data, orient="index")
#     df.to_csv(CSV_OUTPUT_PATH, index=False)
#     print(f"Responses saved to CSV at: {CSV_OUTPUT_PATH}")
# else:
#     print(f"Job failed: {batch_prediction_job.error}")

# print(f"Job output location: {batch_prediction_job.output_location}")

## serch for this string * Creatinine clearance or radioisotope glomerular filtration rate >= 70 mL/min in criteria column

# ravis_df = pd.read_pickle("ravis_df_with_responses.pkl")
for i,row in ravis_df.iterrows():
    if "* WHO/ECOG performance status 0 <= age <= 1" in row["criteria"]:
        print(row["title"])
        print(row["criteria"])
        print(row["description"])
        print("---------------------------------------------------")

An Exploratory Study: Dendritic Cells for Immunotherapy of Metastatic Endometrial Cancer Patients
#Eligibility Criteria:
Inclusion criteria

* women >= 18 years with histologically confirmed stage IV or metastatic carcinoma of the endometrium of the endometroid, serous or carcinosarcoma type.
* Hormone receptor negative or
* resistant to hormonal therapy
* ineligible for hormonal therapy because of other reasons
* eligible for treatment with carboplatin paclitaxel combination chemotherapy
* Life expectancy >= 6 months
* WHO/ECOG performance status 0 <= age <= 1 (Karnofsky index 100 <= age <= 70)
* WBC >2.0 -109/l, neutrophils >1.5 <= age <= 109/L lymphocytes >0.8 <= age <= 109/L, platelets >100 <= age <= 109/L, hemoglobin >5,6 mmol/L (9.0 g/dL), serum creatinine <150 µmol/L, AST/ALT <3 x ULN, serum bilirubin <1.5 x ULN (exception: Gilbert's syndrome is permitted)
* Expression of survivin and/or muc1 on tumor material
* Expected adequacy of follow-up
* Postmenopausal or evidence of non-

In [12]:
import re
import json
import pandas as pd

def adjust_indentation_for_colons(line_objs):
    """
    Adjust indentation for lines that follow a parent line ending with ':'.
    Lines following a line ending with ':' and having the same indentation
    are treated as children by increasing their indentation.
    """
    i = 0
    n = len(line_objs)
    while i < n:
        this_line = line_objs[i]
        text = this_line['text'].rstrip()
        indent = this_line['indent']

        # Check if the current line ends with ':'
        if text.endswith(':'):
            # We'll move forward to see if subsequent lines have the same indent.
            j = i + 1
            while j < n:
                next_line = line_objs[j]
                next_text = next_line['text']
                next_indent = next_line['indent']

                if next_text.endswith(':'):
                    # Next line has its own child block, stop bumping here
                    break
                if next_indent < indent:
                    # Once we find indentation smaller, we stop
                    break
                if next_indent == indent:
                    # Same indent => treat it as child => artificially bump
                    line_objs[j]['indent'] = indent + 2  # Increase indentation
                j += 1

        i += 1

    return line_objs

def parse_lines_to_tree(lines):
    """
    Build a tree from a list of lines (each a dict with `indent` and `text`).
    Each node = { 'text': str, 'indent': int, 'children': [...] }.
    """
    # Initialize root sentinel
    root = {'text': 'ROOT', 'indent': -1, 'children': []}
    stack = [root]

    for line in lines:
        line_indent = line['indent']
        line_text = line['text']

        # Pop from stack until finding the correct parent
        while stack and stack[-1]['indent'] >= line_indent:
            stack.pop()

        # The current top of the stack is the parent
        parent = stack[-1]
        node = {'text': line_text, 'indent': line_indent, 'children': []}
        parent['children'].append(node)
        stack.append(node)

    return root

def gather_all_texts(node, combined_list):
    """
    Recursively gather text from node and its descendants into combined_list.
    Used when flattening deeper than max_depth.
    """
    combined_list.append(f"{node['text'].strip()}")
    for c in node['children']:
        gather_all_texts(c, combined_list)

def flatten_tree_with_rules(node, current_line, depth, max_depth=3, max_siblings=5):
    """
    Flatten a node in the tree with rules:
      - If depth >= max_depth, aggregate all descendants in one line.
      - If a node has > max_siblings children, also aggregate them in one line.
      - Otherwise, append each child as a separate bullet.
    """
    results = []

    # If over max depth, gather everything as one line
    if depth >= max_depth and node['children']:
        combo = []
        gather_all_texts(node, combo)
        return [f"{current_line} {' '.join(combo)}"]

    # If no children => just return current_line
    if not node['children']:
        return [current_line]

    # If there are children, check how many
    num_children = len(node['children'])
    if num_children > max_siblings and depth > 1:
        # Merge them into a single line
        bullets = []
        for child in node['children']:
            bullets.append(f"{child['text'].strip()}")
        merged = " ".join(bullets)
        return [f"{current_line} {merged}"]

    # Otherwise, flatten each child individually
    for child in node['children']:
        child_text = f"{current_line} {child['text'].strip()}"
        child_flat = flatten_tree_with_rules(
            child,
            child_text,
            depth + 1,
            max_depth=max_depth,
            max_siblings=max_siblings
        )
        results.extend(child_flat)

    return results

def parse_criteria_final(text):
    """
    Parses #Eligibility Criteria blocks from the text.
    For each block, extracts:
      - Inclusion criteria (handling indentation and colons)
      - Exclusion criteria (handling indentation and colons)
      - Sex, Min Age, Max Age, Age Groups, Accepts Healthy Volunteers
    Returns a list of JSON strings, one per #Eligibility Criteria block.
    """
    # Split into sections by "#Eligibility Criteria:"
    eligibility_sections = text.split("#Eligibility Criteria:")
    parsed_results = []

    for section in eligibility_sections:
        section = section.strip()
        if not section:
            continue

        # Split lines and assign indentation
        raw_lines = section.split('\n')
        line_objs = []
        for ln in raw_lines:
            if ln.strip():
                indent_len = len(ln) - len(ln.lstrip())
                line_objs.append({'indent': indent_len, 'text': ln.strip()})

        # Pre-process to handle lines ending with ':'
        line_objs = adjust_indentation_for_colons(line_objs)

        # Build the tree
        root = parse_lines_to_tree(line_objs)

        # Initialize parsed block data
        block_data = {
            "Inclusion Criteria": [],
            "Exclusion Criteria": [],
            "Sex": None,
            "Minimum Age": None,
            "Maximum Age": None,
            "Age Groups": [],
            "Accepts Healthy Volunteers": None
        }

        # Process Inclusion Criteria
        inclusion_match = re.search(
            r"Inclusion\s*criteria\s*([\s\S]*?)(?=Exclusion\s*criteria|##Sex)",
            section,
            re.IGNORECASE
        )
        if inclusion_match:
            inclusion_block_text = inclusion_match.group(1)
            inclusion_lines = inclusion_block_text.split('\n')
            inclusion_line_objs = [{'indent': len(ln) - len(ln.lstrip()), 'text': ln.strip()} for ln in inclusion_lines if ln.strip()]
            inclusion_line_objs = adjust_indentation_for_colons(inclusion_line_objs)
            inclusion_root = parse_lines_to_tree(inclusion_line_objs)
            for top_node in inclusion_root['children']:
                flattened = flatten_tree_with_rules(
                    top_node,
                    top_node['text'].strip(),
                    depth=0,
                )
                block_data["Inclusion Criteria"].extend(flattened)

        # Process Exclusion Criteria
        exclusion_match = re.search(
            r"Exclusion\s*criteria\s*([\s\S]*?)(?=##Sex)",
            section,
            re.IGNORECASE
        )
        if exclusion_match:
            exclusion_block_text = exclusion_match.group(1)
            exclusion_lines = exclusion_block_text.split('\n')
            exclusion_line_objs = [{'indent': len(ln) - len(ln.lstrip()), 'text': ln.strip()} for ln in exclusion_lines if ln.strip()]
            exclusion_line_objs = adjust_indentation_for_colons(exclusion_line_objs)
            exclusion_root = parse_lines_to_tree(exclusion_line_objs)
            for top_node in exclusion_root['children']:
                flattened = flatten_tree_with_rules(
                    top_node,
                    top_node['text'].strip(),
                    depth=0,
                )
                block_data["Exclusion Criteria"].extend(flattened)

        # Extract Sex
        sex_match = re.search(r"##Sex\s*:\s*(.+)", section, re.IGNORECASE)
        if sex_match:
            block_data["Sex"] = sex_match.group(1).strip()

        # Extract Minimum Age
        min_age_match = re.search(r"Minimum\s+Age\s*:\s*([\d\w\s]+)", section, re.IGNORECASE)
        if min_age_match:
            block_data["Minimum Age"] = min_age_match.group(1).strip()

        # Extract Maximum Age
        max_age_match = re.search(r"Maximum\s+Age\s*:\s*([\d\w\s]+)", section, re.IGNORECASE)
        if max_age_match:
            block_data["Maximum Age"] = max_age_match.group(1).strip()

        # Extract Age Groups
        age_group_match = re.search(r"Age Group.*:\s*(.+)", section, re.IGNORECASE)
        if age_group_match:
            raw_groups = age_group_match.group(1)
            age_groups = [g.strip() for g in raw_groups.split(',') if g.strip()]
            block_data["Age Groups"] = age_groups

        # Extract Accepts Healthy Volunteers
        hv_match = re.search(r"##Accepts Healthy Volunteers\s*:\s*(\w+)", section, re.IGNORECASE)
        if hv_match:
            hv_value = hv_match.group(1).strip().lower()
            if hv_value == "yes":
                block_data["Accepts Healthy Volunteers"] = True
            elif hv_value == "no":
                block_data["Accepts Healthy Volunteers"] = False
            else:
                block_data["Accepts Healthy Volunteers"] = None

        # Convert the dictionary to a JSON string
        parsed_results.append(json.dumps(block_data))

    return parsed_results

# ------------------------------------------------------------------------
# Example Usage with Test Cases
# ------------------------------------------------------------------------

if __name__ == "__main__":
    # Combined Test Cases Including the Latest Nested and Colon-based Cases
    all_test_cases_combined = """
#Eligibility Criteria:
Inclusion Criteria:

* Locally unresectable or metastatic carcinoid or pancreatic neuroendocrine tumors. To be classified as having a pancreatic neuroendocrine tumor, patients must have clinical evidence of currently having or having had a primary pancreatic neuroendocrine lesion.
* Measurable disease by RECIST criteria
* Evidence of progressive disease (by RECIST) within 12 months of study entry.
* Tumors must be considered well- or moderately-differentiated. Patients with poorly differentiated neuroendocrine carcinoma of small cell carcinoma are excluded from this study.
* Adequate hepatic, renal, bone marrow and glycemic function as outlined in the protocol
* Prior treatment with chemotherapy, hepatic artery embolization, surgery or other therapeutic agents is allowed.
* Prior or concurrent therapy with somatostatin analogs is permitted: however patients must continue on a stable dose of somatostatin analogs while receiving study treatment.
* 18 years or older
* ECOG performance status 0, 1, or 2 [Eastern Cooperative Oncology Group ]
* Life expectancy of at least 12 weeks
* Negative pregnancy test
* Ability to sign informed consent

Exclusion Criteria:

* Poorly differentiated or small cell neuroendocrine carcinomas
* Insulin secreting pancreatic neuroendocrine tumors (insulinomas)
* Clinically apparent central nervous system metastases or carcinomatous meningitis.
* Myocardial infraction in the past 6 months
* Major surgery 4 weeks prior to enrollment
* Uncontrolled serious medical or psychiatric illness
* Pregnant or lactating women. Both men and women of childbearing potential must be advised of the importance of using effective birth control measures during the course of the study.
* Prior antitumor therapy within 4 weeks of enrollment (with the exception of somatostatin analogs).
* Recent infection requiring systemic anti-infective treatment that was completed 14 days or less prior to enrollment (with the exception of uncomplicated urinary tract infection or upper respiratory tract infection).
* Known positive test for human immunodeficiency virus, hepatitis C, chronic or active hepatitis B
* Prior IGF or IGF receptor inhibitor therapy [insulin like growth factor ]
##Sex :
ALL
##Ages : 
- Minimum Age : 18 Years
- Age Group (Child: birth-17, Adult: 18-64, Older Adult: 65+) : ADULT, OLDER_ADULT

##Accepts Healthy Volunteers:
 No


#Eligibility Criteria:
Inclusion Criteria:

* patients undergoing pancreatic resection for a presumed, although not necessarily biopsy-proven pancreatic malignancy,
* ages 18 years to 80 years

Exclusion Criteria:

* Pregnant women
##Sex :
ALL
##Ages : 
- Minimum Age : 18 Years
- Maximum Age : 80 Years
- Age Group (Child: birth-17, Adult: 18-64, Older Adult: 65+) : ADULT, OLDER_ADULT

##Accepts Healthy Volunteers:
 No


#Eligibility Criteria:
Inclusion criteria

* At least 18 years
* A histologically confirmed diagnosis of the following:

   Follicular lymphoma, Grade 1, 2, or 3 or diffuse large cell lymphoma concurrent with or following the diagnosis of follicular lymphoma (World Health Organization/Revised European-American Lymphoma [WHO/REAL] classification).

   International Working Formulation histological equivalents included:

   Follicular, small-cleaved; Follicular, mixed small-cleaved and large-cell; Follicular large-cell; or Transformed diffuse large-cell lymphoma following or concurrent with a diagnosis of follicular lymphoma.
* Stage III or IV disease at the time of study entry (based on Ann Arbor Staging Classification)
* Previously untreated or recurrent lymphoma after no more than 4 prior qualifying therapy regimens; steroids alone, as treatment for lymphoma, not considered a treatment regimen

Exclusion criteria

* Greater than 25% of the intratrabecular marrow space involved by lymphoma in bone marrow biopsy specimens as assessed microscopically within 90 days of study enrollment; a unilateral bone marrow biopsy was adequate; marrow core was greater than or equal to 2.0 cm in length
* Prior chemotherapy, biologic therapy, steroids, or radiation therapy as treatment for NHL within 28 days prior to study enrollment; subjects receiving low doses of steroids for non neoplastic disease acceptable to enter this study ('Low dose steroids' was defined as less than or equal to 10 mg of prednisone or equivalent per day.)
* Prior rituximab therapy within 120 days prior to study enrollment
* Prior radioimmunotherapy
* Prior splenectomy
* Splenomegaly defined as spleen mass greater than 700 grams, where splenic mass was defined as follows:

   Spleen mass = л(X x Y x Z)/6 Where X and Y are the greatest perpendicular diameters in cm on any single CT scan slice, and Z is the number of CT scan slices upon which the spleen is visible times the slice thickness in cm
* Bulky disease as defined as any uni-dimensional measurement of lymphomatous mass exceeding 7 cm
* Prior malignancy other than lymphoma, except for adequately treated skin cancer, in situ cervical cancer, or other cancer for which the subject had a generally accepted risk of recurrence less than 20%
* Central nervous system involvement by lymphoma
* Evidence of active infection requiring IV antibiotics at the time of study enrollment
* Known human immunodeficiency virus (HIV) infection
* New York Heart Association Class III or IV heart disease or other serious illness that would preclude evaluation.
* Active obstructive hydronephrosis
* Evidence of clinically significant ascites or pleural effusion observed on screening physical examination or baseline CT scan
* Prior myeloablative therapy
* History of failed stem cell collection
* Pregnant or nursing subjects (Subjects of childbearing potential had to have a negative serum pregnancy test within 21 days of study enrollment. Males and females of childbearing age had to agree to use effective contraception for up to 12 months after the radioimmunotherapy.)
##Sex :
ALL
##Ages : 
- Minimum Age : 18 Years
- Age Group (Child: birth-17, Adult: 18-64, Older Adult: 65+) : ADULT, OLDER_ADULT

##Accepts Healthy Volunteers:
 No


#Eligibility Criteria:
Inclusion criteria

* Creatinine based on age as follows:

  * <= 0.8 mg/dL for patients age 5 and under
  * <= 1.0 mg/dL for patients age 6 to 10
  * <= 1.2 mg/dL for patients age 11 to 15
  * <= 1.5 mg/dL for patients age 16 to 21

Exclusion criteria

*
##Sex :
ALL
##Ages : 
- Minimum Age : 1 Month
- Maximum Age : 85 Years
- Age Group (Child: birth-17, Adult: 18-64, Older Adult: 65+) : ADULT, CHILD, OLDER_ADULT

##Accepts Healthy Volunteers:
 No


#Eligibility Criteria:
Inclusion criteria

* Meets 1 of the following criteria:

  * Intermediate (i.e., rarely metastasizing) or malignant STS, including any of the following:

    * Adipocytic tumor, including liposarcoma of any of the following histology subtypes:

      * Dedifferentiated
      * Myxoid
      * Round cell
      * Pleomorphic type
      * Mixed-type
      * Not otherwise specified (NOS)
    * Fibroblastic/myofibroblastic tumors, including any of the following:

      * Solitary fibrous tumor
      * Hemangiopericytoma
      * Low-grade myofibroblastic sarcoma
      * Myxoinflammatory fibroblastic sarcoma
      * Adult fibrosarcoma*
      * Myxofibrosarcoma
      * Low-grade fibromyxoid sarcoma or hyalinizing spindle-cell tumor
      * Sclerosing epithelioid fibrosarcoma

Exclusion criteria

*
    """.strip('\n')

    # Parse the combined test cases
    parsed_results = parse_criteria_final(all_test_cases_combined)

    # Create DataFrame with JSON strings in one column
    df = pd.DataFrame({"Eligibility Criteria (JSON)": parsed_results})

    # Display the DataFrame
    print("Eligibility Criteria in JSON Format:")
    print(df.to_json(orient='records', indent=2))


Eligibility Criteria in JSON Format:
[
  {
    "Eligibility Criteria (JSON)":"{\"Inclusion Criteria\": [\": * Locally unresectable or metastatic carcinoid or pancreatic neuroendocrine tumors. To be classified as having a pancreatic neuroendocrine tumor, patients must have clinical evidence of currently having or having had a primary pancreatic neuroendocrine lesion.\", \": * Measurable disease by RECIST criteria\", \": * Evidence of progressive disease (by RECIST) within 12 months of study entry.\", \": * Tumors must be considered well- or moderately-differentiated. Patients with poorly differentiated neuroendocrine carcinoma of small cell carcinoma are excluded from this study.\", \": * Adequate hepatic, renal, bone marrow and glycemic function as outlined in the protocol\", \": * Prior treatment with chemotherapy, hepatic artery embolization, surgery or other therapeutic agents is allowed.\", \": * Prior or concurrent therapy with somatostatin analogs is permitted: however patients m